In [ ]:
#### model domain : single stream:  Length = 1000 m, Width = 1 m, dx = 10 m, dy= 1 m, Q = 0.1 m$^3$/sec
#### Diffusion = 0.2 m$^2$/sec, Dispersion = 0
#### bondary condition : prescribed concentration at the inlet boundary face (x=0 m)
#### initial condition : zero concentration
#### chemsitry: radioactive decay
#### Comparison with PFloTran radioactive decay test at the middle (x=49 m) of the stream 

In [ ]:
import sys, os, math, h5py, csv
import matplotlib.pyplot as plt
from scipy import integrate
import numpy as np
sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils')
import read_helper as read
#sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
#import readdata

In [ ]:
path = '/Users/ajc/FUSE/simulations/tests/ats-demos/13_reactive_transport/01_stream'

In [ ]:
# Read PFOTRAN standalone data
pflotran_file = os.path.join(path, 'data/pflotran_out/denitrification_reach.tec')
pflotran_data = dict()
#plfotran_data['time'] = np.loadtxt(pflotran_file,usecols=3,skiprows=3) #time in hours
pflotran_data['X'] = np.loadtxt(pflotran_file,usecols=0,skiprows=3)  # Tracer 1
pflotran_data['DOM1'] = np.loadtxt(pflotran_file,usecols=3,skiprows=3)  # Tracer 1
pflotran_data['O2(aq)'] = np.loadtxt(pflotran_file,usecols=4,skiprows=3)  # Tracer 1
pflotran_data['NO3-'] = np.loadtxt(pflotran_file,usecols=5,skiprows=3)  # Tracer 1


In [ ]:
F= ['srun3C'] # Logical mesh
F=['test1']
Files = [os.path.join(path,f,'visdump_surface_data.h5') for f in F]
vars = ['surface-total_component_concentration', 'surface-total_component_concentration','surface-total_component_concentration']
Suffix = ['.cell.DOM1 conc', '.cell.NO3- conc', '.cell.O2(aq) conc']

In [ ]:
#%matplotlib qt
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
#axs = axs.ravel()

Colors = ['r','k','m']
F_L = ['DOM1', 'NO3', 'O2']
X = np.linspace(0,1000,100)

for i, v in enumerate(vars):

    plt.subplot(111)
    for j,f in enumerate(Files):
        dat = read.GetSurfVarFromVis_All(f,v,suffix=Suffix[i])[v]
        
        data = np.array(dat[-1])
        plt.semilogy(X,1e-3*data,'-',color=Colors[i],label=F_L[i],linewidth=2)
        
        #percentage removal of a species
        #data_pc = [100*(dat[0][0] - d1 )/dat[0][0] for d1 in data]
        #plt.plot(X,data_pc,color=Colors[i],label=F_L[i],linewidth=2)
        
    #plt.ylabel('component removal [%]')    
    plt.ylabel(vars[i] + ' [M]') # Molarity = Moles/liter
    plt.xlabel('Distance [m]')
    
plt.semilogy(pflotran_data['X'],pflotran_data['DOM1'],'k',linestyle='--',label='Pflotran DOM1')
plt.semilogy(pflotran_data['X'], pflotran_data['NO3-'],'g',linestyle='--',label='Pflotran NO3-')
plt.semilogy(pflotran_data['X'], pflotran_data['O2(aq)'],'c',linestyle='--',label='Pflotran O2(aq)')
plt.legend(loc='lower right',fontsize=10)

plt.ylim(1e-6,1e-3)

plt.show()

In [ ]:
outfile="/Users/ajc/Core/PostProcessing/ReactiveTransport/Hyporheic/Coweeta/June4/"

In [ ]:
import seaborn as sns

def subgrid_C(sg_file,sg_var, suff):
        sgC_data = readdata.GetSurfVarFromVis_All(sg_file,sg_var,suffix=suff)
        sgConcentration = sgC_data[sg_var]
        return sgConcentration
    
def plot_surface_subgrid(cycle,suff):

    X = np.linspace(0,3000,60+1,dtype='i')
    X1 = np.linspace(0,len(X),len(X),dtype='i')
    X1 = X1[0:1]
    
    n_taus = 20
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))#,facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.5, wspace=.002)
    plt.tight_layout()
    axs = axs.ravel()
   
    for i,x1 in enumerate(X1):
        for c1, c in enumerate(Suffix):
            sg_infile = os.path.join(path,F[0],'visdump_subgrid_%s_data.h5'%x1)
            
            sg_var='subgrid_%s-total_component_concentration'%x1
            print (sg_var)
            dat1 = subgrid_C(sg_file=sg_infile,sg_var=sg_var, suff=c)
            
            dat11a = 1e-3*np.reshape(dat1,(-1,n_taus))
            #print (dat11a.shape)
            #for k in range(dat11a.shape[0]):
            dat11 = dat11a[cycle]
            print (dat11)
            axs[3*i+c1].plot(dat11,'.',label='sg_%s_%s'%(x1,c))
            axs[3*i+c1].set_xlabel('# cells in Tau space')
            axs[3*i + c1].legend()
            axs[3*i + c1].set_ylim(1e-4,4e-4)
            #plt.legend(loc='best', fontsize=9)
    plt.savefig(outfile + "11-scott.png",dpi=100)
    plt.show()

In [ ]:
plot_surface_subgrid(cycle = 1, suff=Suffix[1])

In [ ]:
subgrid initial condition = 0
Monon equation? --> Half_saturation_constant
if C is low MONOD reaction is first order
if C is high MONOD reaction is zero order